# Import

In [2]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [6]:
df = pd.read_csv('covid_toy.csv')
df.sample(5)

,age,gender,fever,cough,city,has_covid
32,34,Female,101.0,Strong,Delhi,Yes
59,6,Female,104.0,Mild,Kolkata,Yes
36,38,Female,101.0,Mild,Bangalore,No
30,15,Male,101.0,Mild,Delhi,Yes
83,17,Female,104.0,Mild,Kolkata,No


In [11]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [13]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

# Train and Test

In [14]:
from sklearn.model_selection import train_test_split

In [17]:
x_train , x_test , y_train , y_test = train_test_split(df.iloc[:,:-1],df.iloc[:,-1],test_size=0.2)

In [19]:
x_train

,age,gender,fever,cough,city
27,33,Female,102.0,Strong,Delhi
18,64,Female,98.0,Mild,Bangalore
23,80,Female,98.0,Mild,Delhi
66,51,Male,104.0,Mild,Kolkata
22,71,Female,98.0,Strong,Kolkata
...,...,...,...,...,...
77,8,Female,101.0,Mild,Kolkata
31,83,Male,103.0,Mild,Kolkata
40,49,Female,102.0,Mild,Delhi
47,18,Female,104.0,Mild,Bangalore


# 1. Aam Zindgi

* handling fever section as it has null values

In [24]:
si = SimpleImputer()

x_train_fever = si.fit_transform(x_train[['fever']])

x_test_fever = si.fit_transform(x_test[['fever']])

* Handiling ordinal columns like cough

In [25]:
oe = OrdinalEncoder(categories=[['Mild','Strong']])

In [28]:
x_train_cough = oe.fit_transform(x_train[['cough']])

x_test_cough = oe.fit_transform(x_test[['cough']])

* handling gender column

as it is not depanding categorical type we are using nominal encoding for that 
using one hot encoding

In [41]:
ohe = OneHotEncoder(drop='first', sparse_output=False)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])

x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])

In [54]:
x_train_age = x_train.drop(columns=['gender','city','cough','fever'])
x_test_age = x_test.drop(columns=['gender','city','cough','fever'])


x_test_age

,age
87,47
52,47
62,56
4,65
12,25
33,26
16,69
6,14
80,14
45,72


* Now merging all these 

In [45]:
import numpy as np

In [55]:
x_train_transformed = np.concatenate((x_train_age,x_train_gender_city,x_train_fever,x_train_cough),axis=1)
x_test_transformed = np.concatenate((x_test_age,x_test_gender_city,x_test_fever,x_test_cough),axis=1)

x_train_transformed.shape

(80, 7)

# Mentos Jindgi

In [56]:
from sklearn.compose import ColumnTransformer


In [60]:
transformer = ColumnTransformer(transformers=[
    ('trf1',SimpleImputer(),['fever']),
    ('trf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('trf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
    ],remainder='passthrough')

In [63]:
transformer.fit_transform(x_train)
transformer.fit_transform(x_test)

array([[101.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  47.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  47.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  56.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  65.        ],
       [ 99.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  25.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  26.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  69.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  14.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.    